In [1]:
!!pip install ruptures

['Collecting ruptures',
 '  Downloading ruptures-1.1.10-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (9.2 kB)',
 'Requirement already satisfied: numpy in /usr/local/lib/python3.11/dist-packages (from ruptures) (1.26.3)',
 'Collecting scipy (from ruptures)',
 '  Downloading scipy-1.17.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)',
 'Collecting numpy (from ruptures)',
 '  Downloading numpy-2.4.2-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (6.6 kB)',
 'Downloading ruptures-1.1.10-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl (1.3 MB)',
 '\x1b[?25l   \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/1.3 MB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K   \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b1.3/1.3 MB\x1b \x1b19.9 MB/s\x1b eta \x1b0:00:00\x1b',
 '\x1b[?25hDownloading scipy-1.17.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86

In [3]:
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 76.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [5]:
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 82.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [7]:
!pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 53.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [11]:
# ===== Cell 0: 安裝必要套件 =====
!pip install openpyxl ruptures


[notice] A new release of pip is available: 24.2 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [13]:
# ===== Cell 1: 載入必要套件 =====
import pandas as pd
import numpy as np
import glob
import os

# ===== Cell 2: 快速載入器 =====
class QuickLoader:
    """簡化版載入器，直接產生 arduino_features"""
    
    def __init__(self, data_dir='data/raw'):
        self.data_dir = data_dir
        self.sensor_cols = ['MQ2_raw', 'MQ3_raw', 'MQ9_raw', 'MQ135_raw', 'TGS2602_raw']
        
        # R0 校正參考值
        self.R0_reference = {
            'MQ2_raw': 10.0,
            'MQ3_raw': 60.0,
            'MQ9_raw': 9.8,
            'MQ135_raw': 76.63,
            'TGS2602_raw': 2.5
        }
    
    def load_and_process(self):
        """一次性載入並處理到 arduino_features"""
        print("🔧 快速處理模式...")
        
        # 1. 載入所有檔案
        air_data, pineapple_data = self._load_files()
        
        if not pineapple_data:
            print("❌ 找不到鳳梨數據")
            return None
        
        # 2. 計算 Arduino 特徵
        arduino_features = self._calculate_features(air_data, pineapple_data)
        
        print(f"✅ 完成！產生 {len(arduino_features)} 顆鳳梨的特徵")
        return arduino_features
    
    def _load_files(self):
        """載入 Excel 檔案"""
        xlsx_files = glob.glob(os.path.join(self.data_dir, '*.xlsx'))
        print(f"📂 找到 {len(xlsx_files)} 個檔案")
        
        air_data = {}
        pineapple_data = {}
        
        for file_path in xlsx_files:
            filename = os.path.basename(file_path)
            parts = filename.replace('.xlsx', '').split('_')
            
            if len(parts) < 3:
                continue
            
            is_air = 'air' in filename
            
            # 讀取檔案
            try:
                # 嘗試跳過第一行
                df = pd.read_excel(file_path, engine='openpyxl', skiprows=1)
                
                # 檢查是否有正確的欄位
                if 'timestamp_ms' not in df.columns:
                    df = pd.read_excel(file_path, engine='openpyxl')
                
                # 轉換數字型別
                for col in self.sensor_cols + ['timestamp_ms']:
                    if col in df.columns:
                        df[col] = pd.to_numeric(df[col], errors='coerce')
                
                # 清理缺值
                df = df.dropna().reset_index(drop=True)
                
            except Exception as e:
                print(f"⚠️  載入失敗 {filename}: {e}")
                continue
            
            # 分類儲存
            if is_air:
                if len(parts) == 3:
                    # 共用 air
                    date = parts[1]
                    air_data[f"shared_{date}"] = df
                else:
                    # 專屬 air
                    pid = parts[1]
                    date = parts[2]
                    air_data[f"{pid}_{date}"] = df
            else:
                pid = parts[1]
                date = parts[2]
                
                if pid not in pineapple_data:
                    pineapple_data[pid] = {}
                pineapple_data[pid][date] = df
        
        print(f"   ✓ Air: {len(air_data)}, Pineapple: {len(pineapple_data)} 顆")
        return air_data, pineapple_data
    
    def _calculate_features(self, air_data, pineapple_data):
        """計算 Arduino 特徵"""
        arduino_features = {}
        
        for pid, date_dict in pineapple_data.items():
            arduino_features[pid] = {}
            
            for date, pine_df in date_dict.items():
                # 找對應的 air baseline
                air_df = self._get_air(air_data, pid, date)
                
                if air_df is None:
                    print(f"⚠️  {pid}_{date} 找不到 air baseline")
                    continue
                
                # 計算 Delta
                min_len = min(len(pine_df), len(air_df))
                delta_df = pine_df.iloc[:min_len].copy()
                
                for col in self.sensor_cols:
                    if col in pine_df.columns and col in air_df.columns:
                        delta_df[f'{col}_delta'] = (
                            pine_df[col].iloc[:min_len].values - 
                            air_df[col].iloc[:min_len].values
                        )
                
                # 計算 Arduino 特徵
                features_df = delta_df.copy()
                
                for col in self.sensor_cols:
                    if col in delta_df.columns:
                        sensor_name = col.replace('_raw', '')
                        R0 = self.R0_reference.get(col, 10.0)
                        
                        # Rs/R0
                        features_df[f'{sensor_name}_Rs_R0'] = delta_df[col] / R0
                        
                        # Delta Rs/R0
                        delta_col = f'{col}_delta'
                        if delta_col in delta_df.columns:
                            features_df[f'{sensor_name}_delta_Rs_R0'] = delta_df[delta_col] / R0
                        
                        # 移動平均
                        features_df[f'{sensor_name}_ma10'] = (
                            delta_df[col].rolling(window=10, center=True).mean()
                        )
                        
                        # 移動標準差
                        features_df[f'{sensor_name}_std10'] = (
                            delta_df[col].rolling(window=10, center=True).std()
                        )
                
                arduino_features[pid][date] = features_df
                print(f"   ✓ {pid}_{date}: {features_df.shape}")
        
        return arduino_features
    
    def _get_air(self, air_data, pid, date):
        """取得對應的 air baseline"""
        # 專屬 air
        key = f"{pid}_{date}"
        if key in air_data:
            return air_data[key]
        
        # 共用 air
        shared_key = f"shared_{date}"
        if shared_key in air_data:
            return air_data[shared_key]
        
        return None

# 執行載入
loader = QuickLoader('data/raw')
arduino_features = loader.load_and_process()


🔧 快速處理模式...
📂 找到 54 個檔案
   ✓ Air: 25, Pineapple: 8 顆
   ✓ 07_20250203: (900, 50)
   ✓ 07_20250205: (899, 50)
   ✓ 07_20250204: (900, 50)
   ✓ 08_20250204: (900, 50)
   ✓ 08_20250203: (900, 50)
   ✓ 08_20250205: (900, 50)
   ✓ 02_20250202: (900, 50)
   ✓ 02_20250203: (900, 50)
   ✓ 02_20250205: (900, 50)
   ✓ 02_20250204: (900, 50)
   ✓ 06_20250205: (0, 74)
   ✓ 06_20250204: (900, 50)
⚠️  06_20250202 找不到 air baseline
   ✓ 06_20250203: (900, 50)
   ✓ 04_20250205: (899, 54)
   ✓ 04_20250203: (900, 50)
   ✓ 04_20250204: (900, 54)
   ✓ 03_20250204: (900, 54)
   ✓ 03_20250205: (900, 54)
   ✓ 03_20250203: (900, 50)
   ✓ 01_20250202: (900, 50)
   ✓ 01_20250201: (900, 50)
   ✓ 01_20250203: (900, 50)
   ✓ 01_20250204: (900, 50)
   ✓ 01_20250205: (900, 50)
   ✓ 05_20250203: (900, 50)
   ✓ 05_20250204: (900, 50)
   ✓ 05_20250205: (900, 50)
   ✓ 05_20250202: (900, 50)
✅ 完成！產生 8 顆鳳梨的特徵


In [20]:
# ===== Cell 9: 成熟階段定義（無監督學習）=====

import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from scipy import signal
import ruptures as rpt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

class MaturityStageDetector:
    """使用無監督學習定義鳳梨成熟階段"""
    
    def __init__(self, arduino_features):
        self.arduino_features = arduino_features
        self.sensor_cols = ['MQ2', 'MQ3', 'MQ9', 'MQ135', 'TGS2602']
        self.changepoint_results = {}
        self.clustering_results = {}
        self.maturity_labels = {}
        self.output_dir = 'reports/figures/maturity_detection'
        os.makedirs(self.output_dir, exist_ok=True)
    
    def detect_all(self):
        """批次偵測所有鳳梨的成熟階段"""
        print("="*60)
        print("🔍 Step 3: 成熟階段定義（無監督學習）")
        print("="*60)
        
        for pid, date_dict in self.arduino_features.items():
            print(f"\n🍍 處理 Pineapple {pid}...")
            
            combined_df = self._combine_multi_day_data(date_dict, pid)
            if combined_df is None:
                continue
            
            changepoints = self._detect_changepoints(combined_df, pid)
            self.changepoint_results[pid] = changepoints
            
            cluster_labels = self._cluster_patterns(combined_df, pid, n_clusters=3)
            self.clustering_results[pid] = cluster_labels
            
            maturity_stages = self._define_maturity_stages(
                combined_df, changepoints, cluster_labels, pid
            )
            self.maturity_labels[pid] = maturity_stages
            
            self._visualize_maturity_stages(combined_df, changepoints, 
                                            cluster_labels, maturity_stages, pid)
        
        print("\n" + "="*60)
        print("✅ 成熟階段定義完成！")
        print("="*60)
        
        return self.maturity_labels
    
    def _combine_multi_day_data(self, date_dict, pid):
        """合併多天數據"""
        try:
            df_list = []
            day_counter = 0
            
            for date in sorted(date_dict.keys()):
                df = date_dict[date].copy()
                df['day'] = day_counter
                df['date'] = date
                df['time_global'] = day_counter * len(df) + df.index
                df_list.append(df)
                day_counter += 1
            
            combined = pd.concat(df_list, ignore_index=True)
            print(f"   ✓ 合併 {len(df_list)} 天數據，共 {len(combined)} 筆")
            return combined
            
        except Exception as e:
            print(f"   ❌ 合併數據失敗: {e}")
            return None
    
    def _detect_changepoints(self, df, pid):
        """使用 PELT 演算法偵測變化點"""
        print(f"   🔍 偵測變化點...")
        
        try:
            signals_list = []
            for sensor in self.sensor_cols:
                col_name = f'{sensor}_Rs_R0'
                if col_name in df.columns:
                    signal_data = df[col_name].ffill().bfill()
                    signals_list.append(signal_data.values)
            
            if not signals_list:
                print(f"   ⚠️  找不到 Rs/R0 欄位")
                return []
            
            signal_matrix = np.column_stack(signals_list)
            
            # 👈 關鍵修改：增大 pen 值，減少變化點數
            algo = rpt.Pelt(model="l2", min_size=300, jump=10).fit(signal_matrix)  # min_size 300 → 450
            changepoints = algo.predict(pen=50)  # pen 10 → 50（越大越保守）
            
            changepoints = [cp for cp in changepoints if cp < len(df)]
            
            print(f"   ✓ 偵測到 {len(changepoints)} 個變化點: {changepoints}")
            return changepoints
            
        except Exception as e:
            print(f"   ❌ 變化點偵測失敗: {e}")
            return []

    
    def _cluster_patterns(self, df, pid, n_clusters=3):
        """使用 KMeans 找氣味模式"""
        print(f"   🎯 Clustering 分析...")
        
        try:
            feature_cols = [f'{sensor}_Rs_R0' for sensor in self.sensor_cols]
            feature_cols = [col for col in feature_cols if col in df.columns]
            
            if not feature_cols:
                print(f"   ⚠️  找不到特徵欄位")
                return np.zeros(len(df))
            
            X = df[feature_cols].ffill().bfill().values
            
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)
            
            kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
            labels = kmeans.fit_predict(X_scaled)
            
            print(f"   ✓ 分成 {n_clusters} 個群集")
            print(f"      群集分布: {np.bincount(labels)}")
            
            return labels
            
        except Exception as e:
            print(f"   ❌ Clustering 失敗: {e}")
            return np.zeros(len(df))
    
    def _define_maturity_stages(self, df, changepoints, cluster_labels, pid):
        """定義成熟階段"""
        print(f"   📊 定義成熟階段...")
        
        try:
            stages = np.zeros(len(df), dtype=int)
            
            if changepoints:
                prev_cp = 0
                for i, cp in enumerate(changepoints):
                    stages[prev_cp:cp] = i
                    prev_cp = cp
                stages[prev_cp:] = len(changepoints)
            else:
                stages = cluster_labels
            
            stage_names = {
                0: "Stage 0 (早期)",
                1: "Stage 1 (中期)",
                2: "Stage 2 (後期)",
                3: "Stage 3 (過熟)"
            }
            
            unique_stages, counts = np.unique(stages, return_counts=True)
            
            print(f"   ✓ 成熟階段分布:")
            for stage, count in zip(unique_stages, counts):
                stage_name = stage_names.get(stage, f"Stage {stage}")
                percentage = count / len(stages) * 100
                print(f"      - {stage_name}: {count} 筆 ({percentage:.1f}%)")
            
            return stages
            
        except Exception as e:
            print(f"   ❌ 階段定義失敗: {e}")
            return np.zeros(len(df), dtype=int)
    
    def _visualize_maturity_stages(self, df, changepoints, cluster_labels, 
                                   maturity_stages, pid):
        """視覺化成熟階段分析結果"""
        try:
            fig = make_subplots(
                rows=4, cols=1,
                subplot_titles=(
                    f'🍍 Pineapple {pid} - 感測器訊號',
                    f'🔍 變化點偵測結果',
                    f'🎯 Clustering 結果',
                    f'📊 成熟階段定義'
                ),
                vertical_spacing=0.06
            )
            
            colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8']
            x_axis = df['time_global'] if 'time_global' in df.columns else df.index
            
            for i, sensor in enumerate(self.sensor_cols):
                col_name = f'{sensor}_Rs_R0'
                if col_name in df.columns:
                    fig.add_trace(
                        go.Scatter(x=x_axis, y=df[col_name], mode='lines',
                                 name=sensor, line=dict(color=colors[i], width=1.5)),
                        row=1, col=1
                    )
            
            for cp in changepoints:
                fig.add_vline(x=cp, line_dash="dash", line_color="red", 
                             row=1, col=1, opacity=0.5)
            
            avg_signal = np.mean([df[f'{s}_Rs_R0'].fillna(0) for s in self.sensor_cols 
                                  if f'{s}_Rs_R0' in df.columns], axis=0)
            
            fig.add_trace(
                go.Scatter(x=x_axis, y=avg_signal, mode='lines',
                         name='平均訊號', line=dict(color='black', width=2)),
                row=2, col=1
            )
            
            for cp in changepoints:
                fig.add_vline(x=cp, line_dash="dash", line_color="red", 
                             row=2, col=1, line_width=2)
            
            fig.add_trace(
                go.Scatter(x=x_axis, y=cluster_labels, mode='markers',
                         marker=dict(color=cluster_labels, colorscale='Viridis', size=4)),
                row=3, col=1
            )
            
            stage_colors_map = {0: '#90EE90', 1: '#FFD700', 2: '#FFA500', 3: '#FF6347'}
            stage_colors = [stage_colors_map.get(s, '#808080') for s in maturity_stages]
            
            fig.add_trace(
                go.Scatter(x=x_axis, y=maturity_stages, mode='markers',
                         marker=dict(color=stage_colors, size=4)),
                row=4, col=1
            )
            
            fig.update_layout(height=1200, 
                            title_text=f"🔬 成熟階段分析 - Pineapple {pid}",
                            hovermode='x unified')
            
            filename = f"pineapple_{pid}_maturity_stages.html"
            filepath = os.path.join(self.output_dir, filename)
            fig.write_html(filepath)
            print(f"   💾 已儲存: {filepath}")
            
        except Exception as e:
            print(f"   ❌ 視覺化失敗: {e}")

# 執行
detector = MaturityStageDetector(arduino_features)
maturity_labels = detector.detect_all()

print("\n📋 成熟階段標籤摘要:")
for pid, stages in maturity_labels.items():
    print(f"   🍍 Pineapple {pid}: {len(stages)} 個時間點已標記")


🔍 Step 3: 成熟階段定義（無監督學習）

🍍 處理 Pineapple 07...
   ✓ 合併 3 天數據，共 2699 筆
   🔍 偵測變化點...
   ✓ 偵測到 7 個變化點: [300, 600, 900, 1200, 1500, 1800, 2170]
   🎯 Clustering 分析...
   ✓ 分成 3 個群集
      群集分布: [900 899 900]
   📊 定義成熟階段...
   ✓ 成熟階段分布:
      - Stage 0 (早期): 300 筆 (11.1%)
      - Stage 1 (中期): 300 筆 (11.1%)
      - Stage 2 (後期): 300 筆 (11.1%)
      - Stage 3 (過熟): 300 筆 (11.1%)
      - Stage 4: 300 筆 (11.1%)
      - Stage 5: 300 筆 (11.1%)
      - Stage 6: 370 筆 (13.7%)
      - Stage 7: 529 筆 (19.6%)
   💾 已儲存: reports/figures/maturity_detection/pineapple_07_maturity_stages.html

🍍 處理 Pineapple 08...
   ✓ 合併 3 天數據，共 2700 筆
   🔍 偵測變化點...
   ✓ 偵測到 8 個變化點: [300, 600, 900, 1200, 1500, 1800, 2100, 2400]
   🎯 Clustering 分析...
   ✓ 分成 3 個群集
      群集分布: [900 900 900]
   📊 定義成熟階段...
   ✓ 成熟階段分布:
      - Stage 0 (早期): 300 筆 (11.1%)
      - Stage 1 (中期): 300 筆 (11.1%)
      - Stage 2 (後期): 300 筆 (11.1%)
      - Stage 3 (過熟): 300 筆 (11.1%)
      - Stage 4: 300 筆 (11.1%)
      - Stage 5: 300 筆 (11.1%)
     

In [21]:
#✅ 已完成部分
# Change-point Detection：找出訊號轉折點

# Clustering：找出典型氣味模式

# 自動標記：產生初步的成熟階段標籤（pseudo-labels）

# ⚠️ 還需要做的（關鍵！）
# 根據你的計畫：「結合分析結果與人工判斷，定義成熟階段」

In [24]:
# ===== Cell 10: 自動化品質檢查與儲存（修正版）=====

import json
import pickle
from datetime import datetime

class AutomatedQualityChecker:
    """全自動品質檢查與標籤驗證"""
    
    def __init__(self, maturity_labels, arduino_features, changepoint_results, clustering_results):
        self.maturity_labels = maturity_labels
        self.arduino_features = arduino_features
        self.changepoint_results = changepoint_results
        self.clustering_results = clustering_results
        self.quality_report = {}
        self.output_dir = 'data/processed'
        os.makedirs(self.output_dir, exist_ok=True)
    
    @staticmethod
    def convert_numpy_types(obj):
        """遞迴轉換 numpy 類型為 Python 原生類型"""
        if isinstance(obj, dict):
            return {AutomatedQualityChecker.convert_numpy_types(k): 
                    AutomatedQualityChecker.convert_numpy_types(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [AutomatedQualityChecker.convert_numpy_types(item) for item in obj]
        elif isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return obj
    
    def run_full_check(self):
        """執行完整的自動化檢查流程"""
        print("="*60)
        print("🤖 自動化品質檢查")
        print("="*60)
        
        self._check_label_distribution()
        self._check_changepoint_quality()
        self._check_clustering_quality()
        self._auto_fix_issues()
        self._generate_quality_report()
        self._save_final_labels()
        
        print("\n" + "="*60)
        print("✅ Step 3 完全自動化完成！")
        print("="*60)
        
        return self.maturity_labels
    
    def _check_label_distribution(self):
        """檢查標籤分布是否合理"""
        print("\n📊 檢查標籤分布...")
        
        for pid, stages in self.maturity_labels.items():
            unique, counts = np.unique(stages, return_counts=True)
            distribution = {int(k): int(v) for k, v in zip(unique, counts)}  # 👈 轉換類型
            
            total = len(stages)
            probs = counts / total
            entropy = -np.sum(probs * np.log2(probs + 1e-10))
            max_entropy = np.log2(len(unique))
            normalized_entropy = float(entropy / max_entropy) if max_entropy > 0 else 0.0
            
            issues = []
            if len(unique) == 1:
                issues.append("只有單一階段")
            
            max_ratio = max(probs)
            if max_ratio > 0.9:
                issues.append(f"階段分布過於集中({max_ratio*100:.1f}%)")
            
            if normalized_entropy < 0.3:
                issues.append(f"分布不均勻(熵={normalized_entropy:.2f})")
            
            self.quality_report[pid] = {
                'distribution': distribution,
                'entropy': normalized_entropy,
                'issues': issues,
                'status': 'PASS' if not issues else 'WARNING'
            }
            
            status_icon = "✅" if not issues else "⚠️"
            print(f"   {status_icon} Pineapple {pid}: 階段數={len(distribution)}, 熵={normalized_entropy:.2f}")
    
    def _check_changepoint_quality(self):
        """檢查變化點偵測品質"""
        print("\n🔍 檢查變化點品質...")
        
        for pid, changepoints in self.changepoint_results.items():
            n_points = len(self.maturity_labels[pid])
            n_changepoints = len(changepoints)
            
            cp_density = float(n_changepoints / (n_points / 1000)) if n_points > 0 else 0.0
            
            if n_changepoints == 0:
                status = "無變化點（使用 Clustering）"
            elif cp_density < 0.5:
                status = "變化點稀疏（穩定）"
            elif cp_density > 5:
                status = "變化點密集（可能過擬合）"
            else:
                status = "正常"
            
            if pid in self.quality_report:
                self.quality_report[pid]['changepoint_density'] = cp_density
                self.quality_report[pid]['changepoint_status'] = status
            
            print(f"   🍍 {pid}: {n_changepoints} 個變化點, {status}")
    
    def _check_clustering_quality(self):
        """檢查 Clustering 品質"""
        print("\n🎯 檢查 Clustering 品質...")
        
        from sklearn.metrics import silhouette_score
        
        for pid, cluster_labels in self.clustering_results.items():
            if pid not in self.arduino_features:
                continue
            
            try:
                all_features = []
                for date, df in self.arduino_features[pid].items():
                    feature_cols = [f'{s}_Rs_R0' for s in ['MQ2', 'MQ3', 'MQ9', 'MQ135', 'TGS2602']]
                    feature_cols = [col for col in feature_cols if col in df.columns]
                    if feature_cols:
                        X = df[feature_cols].ffill().bfill().values
                        all_features.append(X)
                
                if all_features:
                    X_combined = np.vstack(all_features)
                    
                    if len(np.unique(cluster_labels)) > 1:
                        score = float(silhouette_score(X_combined, cluster_labels))
                        
                        if score > 0.5:
                            quality = "優秀"
                        elif score > 0.3:
                            quality = "良好"
                        elif score > 0.1:
                            quality = "可接受"
                        else:
                            quality = "較差"
                        
                        if pid in self.quality_report:
                            self.quality_report[pid]['silhouette_score'] = score
                            self.quality_report[pid]['clustering_quality'] = quality
                        
                        print(f"   🍍 {pid}: Silhouette={score:.3f}, 品質={quality}")
                    
            except Exception as e:
                print(f"   ⚠️  {pid}: 無法計算 ({e})")
    
    def _auto_fix_issues(self):
        """自動修正檢測到的問題"""
        print("\n🔧 自動修正...")
        
        fixed_count = 0
        for pid, report in self.quality_report.items():
            if report['status'] == 'WARNING' and "只有單一階段" in report['issues']:
                if pid in self.clustering_results:
                    self.maturity_labels[pid] = self.clustering_results[pid]
                    print(f"   🔧 {pid}: 使用 Clustering 結果替代")
                    fixed_count += 1
                    report['status'] = 'FIXED'
        
        if fixed_count > 0:
            print(f"   ✅ 已自動修正 {fixed_count} 個問題")
        else:
            print(f"   ✅ 無需修正")
    
    def _generate_quality_report(self):
        """產生品質報告"""
        print("\n📋 產生品質報告...")
        
        report_path = os.path.join(self.output_dir, 'step3_quality_report.json')
        
        # 轉換所有 numpy 類型
        quality_report_clean = self.convert_numpy_types(self.quality_report)
        
        report_summary = {
            'timestamp': datetime.now().isoformat(),
            'total_pineapples': len(self.maturity_labels),
            'pineapple_reports': quality_report_clean,
            'overall_status': 'PASS' if all(r['status'] in ['PASS', 'FIXED'] 
                                             for r in quality_report_clean.values()) else 'WARNING'
        }
        
        with open(report_path, 'w', encoding='utf-8') as f:
            json.dump(report_summary, f, indent=2, ensure_ascii=False)
        
        print(f"   💾 品質報告已儲存: {report_path}")
        
        pass_count = sum(1 for r in quality_report_clean.values() if r['status'] in ['PASS', 'FIXED'])
        print(f"   📊 摘要: {pass_count}/{len(quality_report_clean)} 顆鳳梨通過檢查")
    
    def _save_final_labels(self):
        """儲存最終標籤"""
        print("\n💾 儲存最終標籤...")
        
        # Pickle
        pkl_path = os.path.join(self.output_dir, 'maturity_labels.pkl')
        with open(pkl_path, 'wb') as f:
            pickle.dump(self.maturity_labels, f)
        print(f"   ✅ Pickle: {pkl_path}")
        
        # JSON
        json_data = self.convert_numpy_types({
            pid: {
                'stages': stages.tolist() if isinstance(stages, np.ndarray) else list(stages),
                'n_samples': len(stages),
                'unique_stages': np.unique(stages).tolist()
            }
            for pid, stages in self.maturity_labels.items()
        })
        
        json_path = os.path.join(self.output_dir, 'maturity_labels.json')
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, indent=2, ensure_ascii=False)
        print(f"   ✅ JSON: {json_path}")
        
        # CSV
        csv_path = os.path.join(self.output_dir, 'maturity_labels_summary.csv')
        summary_data = []
        for pid, stages in self.maturity_labels.items():
            unique, counts = np.unique(stages, return_counts=True)
            for stage, count in zip(unique, counts):
                summary_data.append({
                    'pineapple_id': pid,
                    'stage': int(stage),
                    'count': int(count),
                    'percentage': f"{count/len(stages)*100:.1f}%"
                })
        
        df_summary = pd.DataFrame(summary_data)
        df_summary.to_csv(csv_path, index=False, encoding='utf-8-sig')
        print(f"   ✅ CSV: {csv_path}")

# 執行
checker = AutomatedQualityChecker(
    maturity_labels, 
    arduino_features, 
    detector.changepoint_results,
    detector.clustering_results
)
final_labels = checker.run_full_check()

print(f"\n🎉 Step 3 完成！可以進入 Step 4（特徵工程）")


🤖 自動化品質檢查

📊 檢查標籤分布...
   ✅ Pineapple 07: 階段數=8, 熵=0.99
   ✅ Pineapple 08: 階段數=9, 熵=1.00
   ✅ Pineapple 02: 階段數=4, 熵=1.00
   ✅ Pineapple 06: 階段數=2, 熵=1.00
   ✅ Pineapple 04: 階段數=4, 熵=0.95
   ✅ Pineapple 03: 階段數=3, 熵=1.00
   ✅ Pineapple 01: 階段數=5, 熵=1.00
   ✅ Pineapple 05: 階段數=6, 熵=0.96

🔍 檢查變化點品質...
   🍍 07: 7 個變化點, 正常
   🍍 08: 8 個變化點, 正常
   🍍 02: 3 個變化點, 正常
   🍍 06: 1 個變化點, 正常
   🍍 04: 3 個變化點, 正常
   🍍 03: 2 個變化點, 正常
   🍍 01: 4 個變化點, 正常
   🍍 05: 5 個變化點, 正常

🎯 檢查 Clustering 品質...
   🍍 07: Silhouette=0.646, 品質=優秀
   🍍 08: Silhouette=0.741, 品質=優秀
   🍍 02: Silhouette=0.697, 品質=優秀
   🍍 06: Silhouette=0.501, 品質=優秀
   🍍 04: Silhouette=0.937, 品質=優秀
   🍍 03: Silhouette=0.940, 品質=優秀
   🍍 01: Silhouette=0.732, 品質=優秀
   🍍 05: Silhouette=0.695, 品質=優秀

🔧 自動修正...
   ✅ 無需修正

📋 產生品質報告...
   💾 品質報告已儲存: data/processed/step3_quality_report.json
   📊 摘要: 8/8 顆鳳梨通過檢查

💾 儲存最終標籤...
   ✅ Pickle: data/processed/maturity_labels.pkl
   ✅ JSON: data/processed/maturity_labels.json
   ✅ CSV: data/processed/maturity_

In [25]:
# ===== 分析階段分布 =====

import pandas as pd
import numpy as np

print("="*60)
print("📊 階段分布詳細分析")
print("="*60)

# 載入剛才儲存的標籤
import pickle
with open('data/processed/maturity_labels.pkl', 'rb') as f:
    labels = pickle.load(f)

all_stage_counts = {}

for pid, stages in labels.items():
    unique, counts = np.unique(stages, return_counts=True)
    
    print(f"\n🍍 Pineapple {pid} ({len(stages)} 筆資料):")
    print(f"   階段數: {len(unique)}")
    
    for stage, count in zip(unique, counts):
        percentage = count / len(stages) * 100
        print(f"   Stage {stage}: {count:4d} 筆 ({percentage:5.1f}%)")
        
        # 累計到全局統計
        if stage not in all_stage_counts:
            all_stage_counts[stage] = 0
        all_stage_counts[stage] += count

# 全局統計
print("\n" + "="*60)
print("📊 全局階段分布（所有鳳梨合計）")
print("="*60)

total_samples = sum(all_stage_counts.values())
sorted_stages = sorted(all_stage_counts.items())

for stage, count in sorted_stages:
    percentage = count / total_samples * 100
    print(f"Stage {stage:2d}: {count:5d} 筆 ({percentage:5.1f}%)")

print(f"\n總樣本數: {total_samples}")
print(f"總階段數: {len(all_stage_counts)}")

# 找出樣本數最少的階段
min_stage = min(sorted_stages, key=lambda x: x[1])
max_stage = max(sorted_stages, key=lambda x: x[1])

print(f"\n⚠️  樣本最少階段: Stage {min_stage[0]} ({min_stage[1]} 筆, {min_stage[1]/total_samples*100:.1f}%)")
print(f"✅ 樣本最多階段: Stage {max_stage[0]} ({max_stage[1]} 筆, {max_stage[1]/total_samples*100:.1f}%)")

# 檢查是否有嚴重不平衡
imbalance_ratio = max_stage[1] / min_stage[1]
print(f"\n📊 不平衡比例: {imbalance_ratio:.1f}:1")

if imbalance_ratio > 10:
    print("⚠️  階段分布嚴重不平衡！")
elif imbalance_ratio > 5:
    print("⚠️  階段分布略有不平衡")
else:
    print("✅ 階段分布相對均衡")


📊 階段分布詳細分析

🍍 Pineapple 07 (2699 筆資料):
   階段數: 8
   Stage 0:  300 筆 ( 11.1%)
   Stage 1:  300 筆 ( 11.1%)
   Stage 2:  300 筆 ( 11.1%)
   Stage 3:  300 筆 ( 11.1%)
   Stage 4:  300 筆 ( 11.1%)
   Stage 5:  300 筆 ( 11.1%)
   Stage 6:  370 筆 ( 13.7%)
   Stage 7:  529 筆 ( 19.6%)

🍍 Pineapple 08 (2700 筆資料):
   階段數: 9
   Stage 0:  300 筆 ( 11.1%)
   Stage 1:  300 筆 ( 11.1%)
   Stage 2:  300 筆 ( 11.1%)
   Stage 3:  300 筆 ( 11.1%)
   Stage 4:  300 筆 ( 11.1%)
   Stage 5:  300 筆 ( 11.1%)
   Stage 6:  300 筆 ( 11.1%)
   Stage 7:  300 筆 ( 11.1%)
   Stage 8:  300 筆 ( 11.1%)

🍍 Pineapple 02 (3600 筆資料):
   階段數: 4
   Stage 0:  900 筆 ( 25.0%)
   Stage 1:  900 筆 ( 25.0%)
   Stage 2:  900 筆 ( 25.0%)
   Stage 3:  900 筆 ( 25.0%)

🍍 Pineapple 06 (1800 筆資料):
   階段數: 2
   Stage 0:  900 筆 ( 50.0%)
   Stage 1:  900 筆 ( 50.0%)

🍍 Pineapple 04 (2699 筆資料):
   階段數: 4
   Stage 0:  900 筆 ( 33.3%)
   Stage 1:  900 筆 ( 33.3%)
   Stage 2:  300 筆 ( 11.1%)
   Stage 3:  599 筆 ( 22.2%)

🍍 Pineapple 03 (2700 筆資料):
   階段數: 3
   St

In [26]:
# ===== 重新映射階段 =====

def remap_stages_to_maturity_levels(labels, n_levels=4):
    """
    將細分的階段重新映射到成熟度等級
    
    n_levels=3: 未熟(0) / 成熟(1) / 過熟(2)
    n_levels=4: 未熟(0) / 初熟(1) / 成熟(2) / 過熟(3)
    """
    remapped_labels = {}
    
    for pid, stages in labels.items():
        max_stage = stages.max()
        
        # 根據百分位數重新映射
        if n_levels == 3:
            # 3 等級：0-33% → 0, 33-66% → 1, 66-100% → 2
            new_stages = np.zeros_like(stages)
            new_stages[stages <= max_stage * 0.33] = 0
            new_stages[(stages > max_stage * 0.33) & (stages <= max_stage * 0.67)] = 1
            new_stages[stages > max_stage * 0.67] = 2
            
        elif n_levels == 4:
            # 4 等級：均分
            new_stages = np.zeros_like(stages)
            new_stages[stages <= max_stage * 0.25] = 0
            new_stages[(stages > max_stage * 0.25) & (stages <= max_stage * 0.50)] = 1
            new_stages[(stages > max_stage * 0.50) & (stages <= max_stage * 0.75)] = 2
            new_stages[stages > max_stage * 0.75] = 3
        
        remapped_labels[pid] = new_stages
    
    return remapped_labels

# 重新映射成 4 個等級
maturity_levels = remap_stages_to_maturity_levels(final_labels, n_levels=4)

# 檢查新的分布
print("📊 重新映射後的階段分布（4 個成熟度等級）\n")
all_counts = {}
for pid, stages in maturity_levels.items():
    unique, counts = np.unique(stages, return_counts=True)
    print(f"🍍 {pid}: {dict(zip(unique.astype(int), counts.astype(int)))}")
    
    for stage, count in zip(unique, counts):
        if stage not in all_counts:
            all_counts[stage] = 0
        all_counts[stage] += count

print(f"\n全局分布: {all_counts}")

# 儲存重新映射後的標籤
import pickle
with open('data/processed/maturity_levels_4class.pkl', 'wb') as f:
    pickle.dump(maturity_levels, f)

print("\n✅ 已儲存 4 等級標籤: data/processed/maturity_levels_4class.pkl")


📊 重新映射後的階段分布（4 個成熟度等級）

🍍 07: {np.int64(0): np.int64(600), np.int64(1): np.int64(600), np.int64(2): np.int64(600), np.int64(3): np.int64(899)}
🍍 08: {np.int64(0): np.int64(900), np.int64(1): np.int64(600), np.int64(2): np.int64(600), np.int64(3): np.int64(600)}
🍍 02: {np.int64(0): np.int64(900), np.int64(1): np.int64(900), np.int64(2): np.int64(900), np.int64(3): np.int64(900)}
🍍 06: {np.int64(0): np.int64(900), np.int64(3): np.int64(900)}
🍍 04: {np.int64(0): np.int64(900), np.int64(1): np.int64(900), np.int64(2): np.int64(300), np.int64(3): np.int64(599)}
🍍 03: {np.int64(0): np.int64(900), np.int64(1): np.int64(900), np.int64(3): np.int64(900)}
🍍 01: {np.int64(0): np.int64(1800), np.int64(1): np.int64(900), np.int64(2): np.int64(900), np.int64(3): np.int64(900)}
🍍 05: {np.int64(0): np.int64(900), np.int64(1): np.int64(900), np.int64(2): np.int64(900), np.int64(3): np.int64(900)}

全局分布: {np.int64(0): np.int64(7800), np.int64(1): np.int64(5700), np.int64(2): np.int64(4200), np.int64(3):

In [27]:
# ===== 詳細分析重新映射結果 =====

import numpy as np

# 計算全局統計
total = 7800 + 5700 + 4200 + 6598
print("="*60)
print("📊 4等級成熟度分布分析")
print("="*60)

stage_names = {
    0: "未熟 (Unripe)",
    1: "初熟 (Early Ripe)", 
    2: "成熟 (Ripe)",
    3: "過熟/後期 (Overripe/Late)"
}

distribution = {
    0: 7800,
    1: 5700,
    2: 4200,
    3: 6598
}

for stage, count in distribution.items():
    percentage = count / total * 100
    bar = "█" * int(percentage / 2)
    print(f"\n{stage_names[stage]:20s}")
    print(f"  樣本數: {count:5d} ({percentage:5.1f}%)")
    print(f"  {bar}")

# 計算不平衡比例
max_count = max(distribution.values())
min_count = min(distribution.values())
imbalance = max_count / min_count

print(f"\n{'='*60}")
print(f"📊 總樣本數: {total}")
print(f"📊 不平衡比例: {imbalance:.2f}:1 (最多/最少)")

# 評估
print(f"\n{'='*60}")
print("✅ 品質評估:")

if imbalance < 2:
    print("  ✅ 分布非常均衡 (< 2:1)")
elif imbalance < 3:
    print("  ✅ 分布良好 (< 3:1)")
elif imbalance < 5:
    print("  ⚠️  略有不平衡 (< 5:1)")
else:
    print("  ❌ 嚴重不平衡 (> 5:1)")

# 檢查問題鳳梨
print(f"\n{'='*60}")
print("⚠️  需要注意的鳳梨:")

problem_pineapples = []

# Pineapple 06: 缺少 Stage 1, 2
print("\n  🍍 Pineapple 06: 只有 2 個階段 (0, 3)")
print("     → 原因: 只有 2 天數據，缺少中間過渡階段")
print("     → 建議: 可能需要排除或標記為「不完整」")

# Pineapple 03: 缺少 Stage 2
print("\n  🍍 Pineapple 03: 缺少 Stage 2")
print("     → 可能原因: 從初熟直接跳到過熟")
print("     → 影響: 訓練 Stage 2 分類器時無此鳳梨樣本")

# Pineapple 01: Stage 0 特別多
print("\n  🍍 Pineapple 01: Stage 0 佔比特高 (1800/4500 = 40%)")
print("     → 原因: 記錄了 5 天，前期停留時間長")
print("     → 影響: 可能導致模型偏好預測 Stage 0")

print(f"\n{'='*60}")
print("🎯 建議:")
print("  1. ✅ 可以進入 Step 4 (特徵工程)")
print("  2. ⚠️  考慮排除 Pineapple 06 (資料不完整)")
print("  3. ✅ Stage 2 樣本雖少但仍可用")
print("  4. 📊 追加測試時，重點補充 Stage 2 (成熟期) 樣本")


📊 4等級成熟度分布分析

未熟 (Unripe)         
  樣本數:  7800 ( 32.1%)
  ████████████████

初熟 (Early Ripe)     
  樣本數:  5700 ( 23.5%)
  ███████████

成熟 (Ripe)           
  樣本數:  4200 ( 17.3%)
  ████████

過熟/後期 (Overripe/Late)
  樣本數:  6598 ( 27.2%)
  █████████████

📊 總樣本數: 24298
📊 不平衡比例: 1.86:1 (最多/最少)

✅ 品質評估:
  ✅ 分布非常均衡 (< 2:1)

⚠️  需要注意的鳳梨:

  🍍 Pineapple 06: 只有 2 個階段 (0, 3)
     → 原因: 只有 2 天數據，缺少中間過渡階段
     → 建議: 可能需要排除或標記為「不完整」

  🍍 Pineapple 03: 缺少 Stage 2
     → 可能原因: 從初熟直接跳到過熟
     → 影響: 訓練 Stage 2 分類器時無此鳳梨樣本

  🍍 Pineapple 01: Stage 0 佔比特高 (1800/4500 = 40%)
     → 原因: 記錄了 5 天，前期停留時間長
     → 影響: 可能導致模型偏好預測 Stage 0

🎯 建議:
  1. ✅ 可以進入 Step 4 (特徵工程)
  2. ⚠️  考慮排除 Pineapple 06 (資料不完整)
  3. ✅ Stage 2 樣本雖少但仍可用
  4. 📊 追加測試時，重點補充 Stage 2 (成熟期) 樣本
